In [1]:
import json
import pandas as pd

In [2]:
conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/v0_customized_440/without_module_head/conversations.json'
graph_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/v0_customized_440/without_module_head/graph.jsonl'

In [3]:
import re

def find_module_string(text):
    # Regex pattern to find the string starting with "module" and ending with ";"
    # It uses non-greedy matching to find the shortest match
    pattern = r"module.*?;"
    
    # Using re.DOTALL to make the dot match all characters including newline
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group()  # Returns the matched string
    else:
        return None  # No match found

# Example usage
input_text = """
some initial text
module name="example1" \n   attribu\nte="   value";
some other text
module name="example2" attribute="another value";
"""

# Find the first occurrence
result = find_module_string(input_text)
print("Found substring:", result)

# If you need to find all occurrences instead of just the first one, you can use re.findall
all_matches = re.findall(r"module.*?;", input_text, re.DOTALL)
print("All found substrings:", all_matches)

Found substring: module name="example1" 
   attribu
te="   value";
All found substrings: ['module name="example1" \n   attribu\nte="   value";', 'module name="example2" attribute="another value";']


In [4]:
graph = pd.read_json(graph_path, lines=True)
with open(conv_path, 'r') as f:
    conv = json.load(f)


In [5]:
conv[0]['conversations'][0]['value']

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'clk', 'type': 'input port'}, {'id': 1, 'content': 'rst', 'type': 'input port'}, {'id': 2, 'content': 'instruction', 'type': 'input port'}, {'id': 3, 'content': 'op_type', 'type': 'output port'}, {'id': 4, 'content': 'reg1_id', 'type': 'output port'}, {'id': 5, 'content': 'reg2_id', 'type': 'output port'}, {'id': 6, 'content': 'immediate', 'type': 'output port'}, {'id': 7, 'content': 'decode_opcode', 'type': 'submodule'}, {'id': 8, 'content': 'decode_regs', 'type': 'submodule'}, {'id': 9, 'content': 'decode_immediate', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[2, 2, 2, 7, 8, 8, 9], [7, 8, 9, 3, 4, 5, 6]]\n, An instruction decoder module designed for a RISC-V based CPU architecture. It takes an encoded instruction and decodes it into control signals for various CPU operations. The module utilizes separate decoder submodules for different parts of the instruction, such as operation typ

In [6]:
conv[0]['conversations'][1]['value']

'module instr_decoder (\n    input clk,\n    input rst,\n    input [31:0] instruction,\n    output [6:0] op_type,\n    output [4:0] reg1_id,\n    output [4:0] reg2_id,\n    output [11:0] immediate\n);\n\n    // Submodules for decoding parts of the instruction\n    wire [6:0] op_type_decoded;\n    wire [4:0] reg1_decoded, reg2_decoded;\n    wire [11:0] immediate_decoded;\n\n    opcode_decoder decode_opcode (\n        .instruction(instruction),\n        .op_type(op_type_decoded)\n    );\n\n    regs_decoder decode_regs (\n        .instruction(instruction),\n        .reg1_id(reg1_decoded),\n        .reg2_id(reg2_decoded)\n    );\n\n    immediate_decoder decode_immediate (\n        .instruction(instruction),\n        .immediate(immediate_decoded)\n    );\n\n    // Output assignments\n    assign op_type = op_type_decoded;\n    assign reg1_id = reg1_decoded;\n    assign reg2_id = reg2_decoded;\n    assign immediate = immediate_decoded;\n\nendmodule\n\n// Submodule for opcode decoding\nmodule 

In [7]:
head = find_module_string(conv[0]['conversations'][1]['value'])
conv[0]['conversations'][0]['value'] + head

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'clk', 'type': 'input port'}, {'id': 1, 'content': 'rst', 'type': 'input port'}, {'id': 2, 'content': 'instruction', 'type': 'input port'}, {'id': 3, 'content': 'op_type', 'type': 'output port'}, {'id': 4, 'content': 'reg1_id', 'type': 'output port'}, {'id': 5, 'content': 'reg2_id', 'type': 'output port'}, {'id': 6, 'content': 'immediate', 'type': 'output port'}, {'id': 7, 'content': 'decode_opcode', 'type': 'submodule'}, {'id': 8, 'content': 'decode_regs', 'type': 'submodule'}, {'id': 9, 'content': 'decode_immediate', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[2, 2, 2, 7, 8, 8, 9], [7, 8, 9, 3, 4, 5, 6]]\n, An instruction decoder module designed for a RISC-V based CPU architecture. It takes an encoded instruction and decodes it into control signals for various CPU operations. The module utilizes separate decoder submodules for different parts of the instruction, such as operation typ

In [8]:
for i in range(len(conv)):
    module_head = find_module_string(conv[i]['conversations'][1]['value'])
    conv[i]['conversations'][0]['value'] = conv[i]['conversations'][0]['value'] + '\n' + module_head

In [9]:
conv[100]['conversations'][0]['value']

'Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{\'id\': 0, \'content\': \'clk\', \'type\': \'input port\'}, {\'id\': 1, \'content\': \'rst\', \'type\': \'input port\'}, {\'id\': 2, \'content\': \'enable\', \'type\': \'input port\'}, {\'id\': 3, \'content\': \'data_a\', \'type\': \'input port\'}, {\'id\': 4, \'content\': \'data_b\', \'type\': \'input port\'}, {\'id\': 5, \'content\': \'busy\', \'type\': \'output port\'}, {\'id\': 6, \'content\': \'acc_result\', \'type\': \'output port\'}, {\'id\': 7, \'content\': \'u_multiplier\', \'type\': \'submodule\'}, {\'id\': 8, \'content\': \'u_accumulator\', \'type\': \'submodule\'}, {\'id\': 9, \'content\': \'u_busy_signal\', \'type\': \'submodule\'}], \nedge_attrs: [], \nconnectivity: [[3, 4, 2, 0, 1, 7, 2, 9, 8], [7, 7, 8, 8, 8, 8, 9, 5, 6]]\n, Design a modular Verilog system for a 16-bit multiplier-accumulator (MAC). The circuit should sequentially multiply two 8-bit inputs and accumulate the result into a 16-bit register.

In [10]:
new_conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/v0_customized_440/with_module_head/conversations.json'



In [11]:
with open(new_conv_path, 'w') as f:
    json.dump(conv, f)

In [12]:
import json
with open(new_conv_path, 'r') as f:
    new_conv = json.load(f)
    


In [13]:
len(new_conv)

480

In [ ]:
new_conv[100]['conversations'][0]['value']

In [32]:
len(graph)

2148